## 1 The Computational Model

In [ ]:
%matplotlib inline
import cv2
import numpy as np
import matplotlib.pyplot as plt

## 1.1 Topographic feature maps

### 1.1.1
Read `test0.jpg` from $640x480$ directory, and split the image into 3 channels of RGB namely `r`, `g`, `b`. To compute the intensity `I`, use the formula $I = (r + g + b)/3$.

In [ ]:
img = plt.imread('test0.jpg')
r = img[:, :, 0]
g = img[:, :, 1]
b = img[:, :, 2]
I = (r + g + b) / 3

plt.imshow(img)
plt.show()

### 1.1.2
Normalize `r`, `g`, `b` by `I`.

In [ ]:
# def normalizeImage(n, d, t):
#     return np.reshape([i/j if j > t else 0 for i,j in zip(np.nditer(n), np.nditer(d))], d.shape)

In [ ]:
def normalizeImage(nom, denom, threshold):
    nom = np.copy(nom)
    yes = np.where(denom > threshold)
    nom[np.where(denom <= threshold)] = 0
    nom[yes] = nom[yes] / denom[yes]
    return nom

threshold = 0.1*np.max(I)
r = normalizeImage(r, I, threshold)
b = normalizeImage(b, I, threshold)
g = normalizeImage(g, I, threshold)

### 1.1.3
Compute `R`, `G`, `B`, `Y` as mentioned in the paper using:

In [ ]:
R = r - (g + b) / 2
G = g - (r + b) / 2
B = b - (r + g) / 2
Y = (r + g) / 2 - np.abs(r - g) / 2 - b

### 1.1.4
Design Gabor filters using `getGaborKernel()` from OpenCV. Select parameters for kernel size, sigma ($\sigma$), theta ($\theta$) and lambd ($\lambda$) to find appropriate filters for four orientations ($\theta = \{0^{\circ}, 45^{\circ}, 90^{\circ}, 135^{\circ}\}$). We may fix gamma ($\gamma$) and phi ($\phi$) with $0.5$ and $0$ respectively. Save your designed Gabor filters to files.

In [ ]:
gabors = []
for i, theta in enumerate(range(0, 180, 45)):
    gabors.append(cv2.getGaborKernel((12, 16), sigma=1, theta=theta, lambd=10, gamma=.5, psi=0))
    plt.imsave('114_gabor_{}.png'.format(i), gabors[-1], cmap='gray')
    ax = plt.subplot(221+i)
    ax.set_title('theta = {}'.format(theta))
    ax.imshow(gabors[i], cmap='gray')
    ax.axis('off')

### 1.1.5
Convolve the gray image (to convert an image to gray image, use: `cv2.cvtColor()`) with the Gabor filters from previous step using `cv2.filter2D()`. Save your output images from four orientations to files.

In [ ]:
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
gabored = []
for i, gabor in enumerate(gabors):
    gabored.append(cv2.filter2D(img_gray, -1, gabor))
    ax = plt.subplot(221+i)
    ax.imshow(gabored[i], cmap='gray')
    ax.axis('off')
    ax.set_title('theta = {}'.format(range(0, 180, 45)[i]))
    plt.imsave('115_gabored_{}.png'.format(i), gabored[-1], cmap='gray')

### 1.1.6
Now, we are ready to go! Apply cv2.pyDown() to create Gaussian pyramids in eight octaves for all maps that we have created. (The scales mentioned in the paper start from scale zero.)

In [ ]:
def makePyramids(img):
    pyramids = [img] # original image is octave zero
    for i in range(7):
        pyramids.append(cv2.pyrDown(pyramids[-1]))
    return pyramids

Rpyr = makePyramids(R)
Gpyr = makePyramids(G)
Bpyr = makePyramids(B)
Ypyr = makePyramids(Y)
Ipyr = makePyramids(I)
Opyrs = [makePyramids(gabor) for gabor in gabors]

## 1.2 Multi-scale center-surround differences
Set fine scales using center $c$ at $c \in \{2, 3, 4\}$ and coarse scales as surround $s = c + \delta$, with $δ \in \{3, 4\}$. Therefore, we get 6 pairs of $(c, s)$ across different octaves as $\{2, 5\}, \{2, 6\}, \{3, 6\}, \{3, 7\}, \{4, 7\}, \{4, 8\}$.

In [ ]:
cs = np.asarray([(2,5), (2,6), (3,6), (3,7), (4,7), (4,8)])-1

### 1.2.1
Compute $\mathcal{I}(c, s)$, $\mathcal{RG}(c, s)$, $\mathcal{BY}(c, s)$ as defined in equation $(1)$, $(2)$ and $(3)$ as described in the paper. In order to perform operations across the different scales, you can use `cv2.resize()` to linearly interpolate the map to any scale.

$$\mathcal{I}(c,s) = |I(c) \ominus I(s)| \quad (1)$$
$$\mathcal{RG}(c,s) = |(R(c) - G(c)) \ominus (G(s) - R(s))| \quad (2)$$
$$\mathcal{BY}(c, s) = |(B(c) - Y(c)) \ominus (Y(s) - B(s))| \quad (3)$$

In [ ]:
csab = lambda c,s,a,b = np.zeros(8): (a[c]-b[c]) - cv2.resize(-1*b[s]+a[s],a[c].shape[::-1])
Ics = list(csab(c,s,Ipyr) for c,s in cs)
RGcs = list(csab(c,s,Rpyr,Gpyr) for c,s in cs)
BYcs = list(csab(c,s,Bpyr,Ypyr) for c,s in cs)

### 1.2.2
Similarly, compute $O(c,s,\theta)$ as in equation $(4)$.

$$O(c,s,\theta) = |O(c,\theta) \ominus O(s,\theta)| \quad (4)$$

In [ ]:
Otcs = [[csab(c,s,Opyr) for c,s in cs] for Opyr in Opyrs]

### 1.2.3
Now, we get 6 maps for intensity, 12 maps for color and 24 for orientation. Apply normalization $\mathcal{N}(\cdot)$ to each map using the same range for all maps. (Read about normalization and how to do it in the paper!)

In [ ]:
from scipy.signal import argrelextrema
def normalize(img):
    M = np.max(img)
    m = np.mean([m for m in img[argrelextrema(img, np.greater)] if m != M])
    return np.copy(img) * ((M - m) ** 2)

In [ ]:
Ics = [normalize(img) for img in Ics]
RGcs = [normalize(img) for img in RGcs]
BYcs = [normalize(img) for img in BYcs]
Otcs = [[normalize(img) for img in Ocs] for Ocs in Otcs]

## 1.3 Across-scale combinations and normalization
We will combine maps from different scales and modalities.

### 1.3.1
In order to combine maps from different scales we select the scale at $\sigma = 4$. Again, we can use `cv2.resize()` to combine maps by their features as shown in equations $(5)$, $(6)$ and $(7)$.

$$\bar{\mathcal{I}} = \oplus^4_{c=2} \oplus^{c+4}_{s=c+3} \mathcal{N}(I(c,s)) \quad (5)$$

$$\bar{C} = \oplus^4_{c=2} \oplus^{c+4}_{s=c+3} [\mathcal{N}(\mathcal{RG}(c,s)) + \mathcal{N}(\mathcal{BY}(c,s))] \quad (6)$$

$$\bar{O} = \sum_{\theta \in \{0^{\circ},45^{\circ},90^{\circ},135^{\circ}\}} \mathcal{N} \left( \oplus^4_{c=2} \oplus^{c+4}_{s=c+3} \mathcal{N}(O(c,s,\theta)) \right) \quad (6)$$

In [ ]:
def addition(imgs, size):
    imgs = [cv2.resize(img, size[::-1]) for img in imgs]
    return np.sum(imgs, 0)
Ibar = addition(Ics, Ics[3].shape)
Cbar = addition([RGcs[i] + BYcs[i] for i in range(len(RGcs))], RGcs[3].shape)
Obar = np.sum([normalize(addition(Ocs, Ics[3].shape)) for Ocs in Otcs], 0)

### 1.3.2
The final saliency map can be obtained using equation $(9)$. Save the saliency map to a file. (You can up-scale the saliency map to the original image size in order to have it map be of the same size as the image.)

$$S=\frac{1}{3} \left(\mathcal{N}(\bar{\mathcal{I}})+\mathcal{N}(\bar{C})+\mathcal{N}(\bar{O})\right) \quad (8)$$

In [ ]:
S = cv2.resize(1/3 * (normalize(Ibar) + normalize(Cbar) + normalize(Obar)), img.shape[1::-1])
plt.imsave('132_saliency.png', S, cmap='gray')

ax1 = plt.subplot(121)
ax1.set_title('saliency map')
ax1.imshow(S, cmap='gray')
ax1.axis('off')
ax2 = plt.subplot(122)
ax2.set_title('originial image')
ax2.imshow(img)
ax2.axis('off')
plt.show()